In [1]:
import nn
import tree
import preprocess

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('./../../dataset/Wine_Quality_Data.csv')

In [4]:
df = preprocess.preprocess_wine_quality(df)

In [5]:
df.head(10)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,color,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,1,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,1,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,1,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,1,5
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,1,5
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,1,5
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,1,7
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,1,7
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,1,5


In [6]:
data = df.to_numpy()

In [7]:
ff = nn.FeedForward(12, 20, target_cls=6, reg=0.2)

In [8]:
scores = ff.train(data, num_epochs=500, learning_rate=1e-4)

In [9]:
mask = data[:, -1] == 6
scores[mask]

array([[ 1.99363313, -0.2583595 ,  2.32154457, ..., -2.12921345,
         6.        ,  0.43938341],
       [ 2.67237568,  0.05416466,  4.90502399, ..., -2.48418175,
         6.        ,  0.65050201],
       [ 1.47212056, -0.31501841,  0.5621111 , ..., -1.79282612,
         6.        ,  0.46996049],
       ...,
       [-0.04620949, -0.01737991, -0.94019524, ...,  0.01567019,
         6.        ,  0.49499071],
       [-0.83245928, -0.32959694, -0.13325749, ...,  0.20243933,
         6.        ,  0.51904016],
       [-0.22692123,  0.3531557 , -1.86513705, ...,  0.56025561,
         6.        ,  0.48774471]])

In [10]:
meta_tree = tree.DODTree()
meta_tree.train(
    train=data,
    ff_dim=20,
    momentum=0.9,
    max_depth=20,
    target_impurity=0.2,
    num_epochs=500,
    learning_rate=1e-5,
    reg=0.2
)

In [11]:
pred = []
for row in data:
    pred.append(meta_tree.predict(row[:-1]))
pred

In [12]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# accuracy_score(data[:,-1], np.array(pred).astype(int))

train, test = train_test_split(df, test_size=0.2, random_state=42)

meta_tree = tree.DODTree()
meta_tree.train(
    train=train,
    ff_dim=20,
    momentum=0.9,
    max_depth=20,
    target_impurity=0.2,
    num_epochs=500,
    learning_rate=1e-5,
    reg=0.2
)

In [13]:
# test.to_numpy().dtype

In [14]:
pred = []
test = test.to_numpy()
for row in test:
    pred.append(meta_tree.predict(row[:-1].astype('float64')))

accuracy_score(test[:,-1], np.array(pred).astype(int))